In [4]:
import numpy as np
import librosa
import librosa.display
import scipy.linalg
import soundfile as sf
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'librosa'

In [6]:
# Load the audio file
file_path = "C:Users\User\Documents\School-Code\python\testing\audio\10sec Figa De Guine.wav"  # Replace with your audio file
y, sr = librosa.load(file_path, sr=None)  # Load original audio

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 7-8: truncated \UXXXXXXXX escape (601219065.py, line 2)

In [ ]:
# Convert to spectrogram
D = librosa.stft(y)  # Short-time Fourier Transform (STFT)
magnitude, phase = np.abs(D), np.angle(D)  # Get magnitude and phase

In [ ]:
# Apply SVD to the magnitude spectrogram
U, S, Vt = scipy.linalg.svd(magnitude, full_matrices=False)

In [ ]:
# Compression: Keep only top k singular values
energy_threshold = 0.99  # Keep enough singular values to preserve 99% energy
total_energy = np.cumsum(S**2) / np.sum(S**2)
k = np.searchsorted(total_energy, energy_threshold) + 1


In [ ]:
# Reconstruct with top-k singular values
compressed_magnitude = np.dot(U[:, :k], np.dot(np.diag(S[:k]), Vt[:k, :]))

In [ ]:
# Reconstruct the audio using inverse STFT
D_compressed = compressed_magnitude * np.exp(1j * phase)  # Restore phase
y_compressed = librosa.istft(D_compressed)  # Inverse STFT

In [ ]:
# Save the compressed audio
sf.write("compressed_audio.wav", y_compressed, sr)

In [ ]:
# Plot original vs compressed spectrogram
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
librosa.display.specshow(librosa.amplitude_to_db(magnitude, ref=np.max), sr=sr, x_axis="time", y_axis="log")
plt.title("Original Spectrogram")

In [ ]:
plt.subplot(1, 2, 2)
librosa.display.specshow(librosa.amplitude_to_db(compressed_magnitude, ref=np.max), sr=sr, x_axis="time", y_axis="log")
plt.title(f"Compressed Spectrogram (k={k})")

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
print(f"Compression complete! Retained {k} singular values. Compressed audio saved as 'compressed_audio.wav'.")